In [2]:
import numpy as np
import pandas as pd
import toytree
from scipy.optimize import minimize
from scipy.linalg import expm

In [5]:
testdata = [0,0,1,1,0,1,0,1,0,1,1,0]
testtree = toytree.rtree.unittree(ntips = 12)
testtree.draw(tree_style = 'p')

(<toyplot.canvas.Canvas at 0x7f9d3f2ec850>,
 <toytree.Render.ToytreeMark at 0x7f9d3f31e7f0>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="265.0px" viewBox="0 0 350.0 265.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t69d30d4b6f6d415390debff386782fe2"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 r10 r11 0.0 0.5 1.0

In [6]:
def data_to_dict(data):
    """
    Parses data into format that can be used by the cond_like and
    pruningalg functions
    """
    values = [{0:-(i-1),1:i} for i in data]
    keys = list(range(0, len(data), 1))
    valuesdict = dict(zip(keys,values))
    return valuesdict

In [7]:
check = data_to_dict(data = testdata)
check

testtree = testtree.set_node_values('test', values = check)
testtree.get_node_values('test',True,True)

array(['', '', '', '', '', '', '', '', '', '', '', {0: 1, 1: 0},
       {0: 0, 1: 1}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 0, 1: 1},
       {0: 1, 1: 0}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 0, 1: 1},
       {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 1, 1: 0}], dtype=object)

In [8]:
def assign_tip_like_values(tree, data):
    """
    Assigns likelihood values to tree tips
    """
    values = [{0:-(i-1),1:i} for i in data]
    keys = list(range(0, len(data), 1))
    valuesdict = dict(zip(keys,values))
    tree = tree.set_node_values(feature = "likelihood", values = valuesdict)
    return tree

In [9]:
mytree = assign_tip_like_values(tree = testtree, data=testdata)
mytree.get_node_values('likelihood',True,True)

array(['', '', '', '', '', '', '', '', '', '', '', {0: 1, 1: 0},
       {0: 0, 1: 1}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 0, 1: 1},
       {0: 1, 1: 0}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 0, 1: 1},
       {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 1, 1: 0}], dtype=object)

In [10]:
def cond_like(likeleft0, likeleft1, likeright0, likeright1, tL, tR, alpha, beta):
    """
    Calculates conditional likelihood of character states at each node
    """

    Q = np.array([[-alpha, alpha], [beta, -beta]])
    probleft = expm(Q*tL)
    probright = expm(Q*tR)
 
    #ancestor is 0
    left0 = probleft[0, 0] * likeleft0 + probleft[0, 1] * likeleft1
    right0 = probright[0, 0] * likeright0 + probright[0, 1] * likeright1
    like_zero = left0*right0
 
    #ancestor is 1
    left1 = probleft[1, 0] * likeleft0 + probleft[1, 1] * likeleft1
    right1 = probright[1, 0] * likeright0 + probright[1, 1] * likeright1
    like_one = left1*right1
 
    return {0: like_zero, 1: like_one}

In [11]:
def pruning_alg(tree, alpha, beta):
    """
    Runs Felsenstein's pruning algorithm on an input tree, given instantaneous transition
    rates alpha and beta. Assigns likelihood scores for characters states at each node.
    Specifically for binary state model. 
    """
    for node in tree.treenode.traverse("postorder"):
        if len(node.children) == 2:
            child1 = node.children[0]
            child2 = node.children[1]
            likedict = cond_like(likeright0 = child1.likelihood[0],
                                 likeright1 = child1.likelihood[1],
                                 likeleft0 = child2.likelihood[0],
                                 likeleft1 = child2.likelihood[1],
                                 tR = child1.dist,
                                 tL = child2.dist,
                                 alpha = alpha,
                                 beta = beta)
            node.likelihood = likedict

In [12]:
pruning_alg(tree=mytree, alpha=8.0, beta=8.0)
mytree.get_node_values('likelihood',True,True)

array([{0: 0.000245307886852543, 1: 0.00024530788738056156},
       {0: 0.03124994585390318, 1: 0.03125004707869125},
       {0: 0.007849865867389302, 1: 0.007849840683369335},
       {0: 0.24999999986417154, 1: 0.24999999986417148},
       {0: 0.1249970722957881, 1: 0.12500289957041805},
       {0: 0.24999999986417154, 1: 0.24999999986417148},
       {0: 0.031400138011181945, 1: 0.031398688225972075},
       {0: 0.24999997186620612, 1: 0.24999997186620618},
       {0: 0.06279862476529781, 1: 0.06279902770966281},
       {0: 0.1261982465855293, 1: 0.12499708636235714},
       {0: 0.28594871310985875, 1: 0.2164652618870571}, {0: 1, 1: 0},
       {0: 0, 1: 1}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 0, 1: 1},
       {0: 1, 1: 0}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 0, 1: 1},
       {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 1, 1: 0}], dtype=object)

In [42]:
def node_like(x0, likeleft0, likeleft1, likeright0, likeright1, tL, tR, anca):
    
    condlik = cond_like(likeleft0, likeleft1, likeright0, likeright1, tL, tR, x0[0], x0[1])
    
    # get full likelihood
    lik = (1 - anca) * condlik[0] + (anca) * condlik[1]
    
    # I don't understand this part
    if anca in [0., 1.]:
        lik /= 2
    
    return -lik #np.log(lik)

In [ ]:
def node_like2()

In [16]:
mytree2 = node_like(tree=mytree)
mytree2.get_node_values('neglike',True,True)

array([-2.45307887e-04, -3.12499965e-02, -7.84985328e-03, -2.50000000e-01,
       -1.24999986e-01, -2.50000000e-01, -3.13994131e-02, -2.49999972e-01,
       -6.27988262e-02, -1.25597666e-01, -2.51206987e-01, -5.00000000e-01,
       -5.00000000e-01, -5.00000000e-01, -5.00000000e-01, -5.00000000e-01,
       -5.00000000e-01, -5.00000000e-01, -5.00000000e-01, -5.00000000e-01,
       -5.00000000e-01, -5.00000000e-01, -5.00000000e-01])

In [45]:
def model_fit(likeleft0, likeleft1, likeright0, likeright1, tL, tR, anca):
    """
    Find the maximum likelihood estimate of the two
    rate model parameters at each node given the data.
    """
    args = (likeleft0, likeleft1, likeright0, likeright1, tL, tR, anca)
    
    # ML estimate
    estimate = minimize(
        fun=node_like, 
        x0=np.array([1., 1.]),
        args=args,
        method='L-BFGS-B',
        bounds=((0, 10), (0, 10))
    )
    
    score = -1 * node_like(estimate.x, *args)
    result = {
        "alpha": round(estimate.x[0], 3),
        "beta": round(estimate.x[1], 3), 
        "lik": round(score, 3),
        "convergence": estimate.success,
    }
    return result

In [46]:
model_fit(0,1,1,0,5,5,0.5)

{'alpha': 1.0, 'beta': 1.0, 'lik': 0.25, 'convergence': True}

In [51]:
def fit_model_at_nodes(tree):
    tree = tree.set_node_values('alpha')
    tree = tree.set_node_values('beta')
    for node in tree.treenode.traverse("postorder"):
        if len(node.children) == 2:
            child1 = node.children[0]
            child2 = node.children[1]
            model = model_fit(likeright0 = child1.likelihood[0],
                              likeright1 = child1.likelihood[1],
                              likeleft0 = child2.likelihood[0],
                              likeleft1 = child2.likelihood[1],
                              tR = child1.dist,
                              tL = child2.dist,
                              anca = 0.5)
            node.alpha = model['alpha']
            node.beta = model['beta']
    return tree            

In [55]:
mytree3 = fit_model_at_nodes(tree=mytree)
mytree3.get_node_values('beta',True,True)

array(['1.0', '1.0', '1.0', '4.259', '0.0', '4.259', '10.0', '6.763',
       '0.0', '0.0', '10.0', '', '', '', '', '', '', '', '', '', '', '',
       ''], dtype='<U32')

In [71]:
def pruning_alg(tree):
    """
    Runs Felsenstein's pruning algorithm on an input tree, given instantaneous transition
    rates alpha and beta. Assigns likelihood scores for characters states at each node.
    Specifically for binary state model. 
    """
    tree = fit_model_at_nodes(tree)
    for node in tree.treenode.traverse("postorder"):
        if len(node.children) == 2:
            child1 = node.children[0]
            child2 = node.children[1]
            likedict = cond_like(likeright0 = child1.likelihood[0],
                                 likeright1 = child1.likelihood[1],
                                 likeleft0 = child2.likelihood[0],
                                 likeleft1 = child2.likelihood[1],
                                 tR = child1.dist,
                                 tL = child2.dist,
                                 alpha = float(node.alpha),
                                 beta = float(node.beta))
            node.likelihood = likedict

In [74]:
mytree4 = pruning_alg(tree=mytree3)
mytree.get_node_values('likelihood',True,True)

array([{0: 0.000245307886852543, 1: 0.00024530788738056156},
       {0: 0.03124994585390318, 1: 0.03125004707869125},
       {0: 0.007849865867389302, 1: 0.007849840683369335},
       {0: 0.24999999986417154, 1: 0.24999999986417148},
       {0: 0.1249970722957881, 1: 0.12500289957041805},
       {0: 0.24999999986417154, 1: 0.24999999986417148},
       {0: 0.031400138011181945, 1: 0.031398688225972075},
       {0: 0.24999997186620612, 1: 0.24999997186620618},
       {0: 0.06279862476529781, 1: 0.06279902770966281},
       {0: 0.1261982465855293, 1: 0.12499708636235714},
       {0: 0.28594871310985875, 1: 0.2164652618870571}, {0: 1, 1: 0},
       {0: 0, 1: 1}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 0, 1: 1},
       {0: 1, 1: 0}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 0, 1: 1},
       {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 1, 1: 0}], dtype=object)